# Regression Week 5: LASSO (coordinate descent)

In this notebook, you will implement your very own LASSO solver via coordinate descent. You will:
* Write a function to normalize features
* Implement coordinate descent for LASSO
* Explore effects of L1 penalty

# Fire up graphlab create

Make sure you have the latest version of graphlab (>= 1.7)

In [1]:
import pprint
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import graphlab as gl
print gl.version
gl.canvas.set_target('ipynb')
from sklearn import metrics

1.8.1


# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = gl.SFrame('data/kc_house_data.gl/')
# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to int, before using it below
sales['floors'] = sales['floors'].astype(int) 

[INFO] This non-commercial license of GraphLab Create is assigned to bbalaji8@gmail.com and will expire on December 09, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-25328 - Server binary: /usr/local/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1454590786.log
[INFO] GraphLab Server Version: 1.8.1


If we want to do any "feature engineering" like creating new features or adjusting existing ones we should do this directly using the SFrames as seen in the first notebook of Week 2. For this notebook, however, we will work with the existing features.

# Import useful functions from previous notebook

As in Week 2, we convert the SFrame into a 2D Numpy array. Copy and paste `get_num_data()` from the second notebook of Week 2.

In [4]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1 # this is how you add a constant column to an SFrame
    # add the column 'constant' to the front of the features list so that we can extract it along with the others:
    features = ['constant'] + features # this is how you combine two lists
    # select the columns of data_SFrame given by the features list into the SFrame features_sframe (now including constant):
    features_sframe = data_sframe[features]
    
    # the following line will convert the features_SFrame into a numpy matrix:
    feature_matrix = features_sframe.to_numpy()
    # assign the column of data_sframe associated with the output to the SArray output_sarray
    output_sarray = data_sframe[output]
    
    # the following will convert the SArray into a numpy array by first converting it to a list
    output_array = output_sarray.to_numpy()
    return(feature_matrix, output_array)

Also, copy and paste the `predict_output()` function to compute the predictions for an entire matrix of features given the matrix and the weights:

In [5]:
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    predictions = np.dot(feature_matrix, weights)

    return(predictions)

# Normalize features
In the house dataset, features vary wildly in their relative magnitude: `sqft_living` is very large overall compared to `bedrooms`, for instance. As a result, weight for `sqft_living` would be much smaller than weight for `bedrooms`. This is problematic because "small" weights are dropped first as `l1_penalty` goes up. 

To give equal considerations for all features, we need to **normalize features** as discussed in the lectures: we divide each feature by its 2-norm so that the transformed feature has norm 1.

Let's see how we can do this normalization easily with Numpy: let us first consider a small matrix.

In [6]:
X = np.array([[3.,5.,8.],[4.,12.,15.]])
print X

[[  3.   5.   8.]
 [  4.  12.  15.]]


Numpy provides a shorthand for computing 2-norms of each column:

In [7]:
# gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
norms = np.linalg.norm(X, axis=0) 
print norms

[  5.  13.  17.]


To normalize, apply element-wise division:

In [8]:
print X / norms # gives [X[:,0]/norm(X[:,0]), X[:,1]/norm(X[:,1]), X[:,2]/norm(X[:,2])]

[[ 0.6         0.38461538  0.47058824]
 [ 0.8         0.92307692  0.88235294]]


Using the shorthand we just covered, write a short function called `normalize_features(feature_matrix)`, which normalizes columns of a given feature matrix. The function should return a pair `(normalized_features, norms)`, where the second item contains the norms of original features. As discussed in the lectures, we will use these norms to normalize the test data in the same way as we normalized the training data. 

In [9]:
def normalize_features(feature_matrix):
    norms = np.linalg.norm(feature_matrix, axis=0)
    print 'normalize_features: norms:'
    print norms
    
    normalized_features = feature_matrix / norms
    return (normalized_features, norms)

To test the function, run the following:

In [10]:
features, norms = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]))
print features
# should print
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print norms
# should print
# [5.  10.  15.]

normalize_features: norms:
[  5.  10.  15.]
[[ 0.6  0.6  0.6]
 [ 0.8  0.8  0.8]]
[  5.  10.  15.]


# Implementing Coordinate Descent with normalized features

We seek to obtain a sparse set of weights by minimizing the LASSO cost function
```
SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).
```
(By convention, we do not include `w[0]` in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use **coordinate descent**: at each iteration, we will fix all weights but weight `i` and find the value of weight `i` that minimizes the objective. That is, we look for
```
argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]
```
where all weights other than `w[i]` are held to be constant. We will optimize one `w[i]` at a time, circling through the weights multiple times.  
  1. Pick a coordinate `i`
  2. Compute `w[i]` that minimizes the cost function `SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|)`
  3. Repeat Steps 1 and 2 for all coordinates, multiple times

For this notebook, we use **cyclical coordinate descent with normalized features**, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:
```
       ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2
```
where
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].
```

Note that we do not regularize the weight of the constant feature (intercept) `w[0]`, so, for this weight, the update is simply:
```
w[0] = ro[i]
```

## Effect of L1 penalty

Let us consider a simple model with 2 features:

In [11]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = \
    get_numpy_data(sales, simple_features, my_output)

Don't forget to normalize features:

In [12]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)

normalize_features: norms:
[  1.47013605e+02   3.34257264e+05   5.14075870e+02]


We assign some random set of initial weights and inspect the values of `ro[i]`:

In [13]:
weights = np.array([1., 4., 1.])

Use `predict_output()` to make predictions on this data.

In [14]:
prediction = predict_output(simple_feature_matrix, weights)

Compute the values of `ro[i]` for each feature in this simple model, using the formula given above, using the formula:
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]
```

*Hint: You can get a Numpy vector for feature_i using:*
```
simple_feature_matrix[:,i]
```

In [15]:
ro = np.zeros(len(simple_features) + 1) # +1 for intercept
#print simple_feature_matrix
for i in xrange(ro.shape[0]):
    ro[i] = np.sum(simple_feature_matrix[:,i] * \
                   (output - prediction + \
                    weights[i] * simple_feature_matrix[:,i]))
print ro

[ 79400300.03492916  87939470.77299108  80966698.67596565]


***QUIZ QUESTION***

Recall that, whenever `ro[i]` falls between `-l1_penalty/2` and `l1_penalty/2`, the corresponding weight `w[i]` is sent to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of `l1_penalty` **would not** set `w[1]` zero, but **would** set `w[2]` to zero, if we were to take a step in that coordinate? 

In [21]:
#print 5.0 * np.array([0.5, 1.0, 2.0])

[  2.5   5.   10. ]


In [40]:
def searchL1P(l1PRange):
    for l1P in l1PRange:
        print 'l1P: %.4f; l1P / ro[2]:%f' % (l1P, l1P / ro[2])
        print (-l1P / 2 <= ro); print (ro + l1P / 2)
        print (ro <= +l1P / 2); print (l1P / 2 - ro)
        print ' '    

In [41]:
searchL1P(ro[2] * np.array([1.0 / 3, 1.0 / 2, 1.0, 2.0, 3.0]))

l1P: 26988899.5587; l1P / ro[2]:0.333333
[ True  True  True]
[  9.28947498e+07   1.01433921e+08   9.44611485e+07]
[False False False]
[-65905850.25560155 -74445020.99366346 -67472248.89663804]
 
l1P: 40483349.3380; l1P / ro[2]:0.500000
[ True  True  True]
[  9.96419747e+07   1.08181145e+08   1.01208373e+08]
[False False False]
[-59158625.36593774 -67697796.10399966 -60725024.00697424]
 
l1P: 80966698.6760; l1P / ro[2]:1.000000
[ True  True  True]
[  1.19883649e+08   1.28422820e+08   1.21450048e+08]
[False False False]
[-38916950.69694633 -47456121.43500825 -40483349.33798283]
 
l1P: 161933397.3519; l1P / ro[2]:2.000000
[ True  True  True]
[  1.60366999e+08   1.68906169e+08   1.61933397e+08]
[ True False  True]
[ 1566398.6410365  -6972772.09702542        0.        ]
 
l1P: 242900096.0279; l1P / ro[2]:3.000000
[ True  True  True]
[  2.00850348e+08   2.09389519e+08   2.02416747e+08]
[ True  True  True]
[ 42049747.97901931  33510577.24095739  40483349.33798282]
 


In [42]:
searchL1P(ro[2] * np.array([1.9, 2.0, 2.1]))

l1P: 153836727.4843; l1P / ro[2]:1.900000
[ True  True  True]
[  1.56318664e+08   1.64857835e+08   1.57885062e+08]
[False False False]
[ -2481936.29276179 -11021107.03082371  -4048334.93379828]
 
l1P: 161933397.3519; l1P / ro[2]:2.000000
[ True  True  True]
[  1.60366999e+08   1.68906169e+08   1.61933397e+08]
[ True False  True]
[ 1566398.6410365  -6972772.09702542        0.        ]
 
l1P: 170030067.2195; l1P / ro[2]:2.100000
[ True  True  True]
[  1.64415334e+08   1.72954504e+08   1.65981732e+08]
[ True False  True]
[ 5614733.57483478 -2924437.16322714  4048334.93379828]
 


In [54]:
searchL1P(ro[2] * np.array([1.99, 2.0, 2.17, 2.18]))

l1P: 161123730.3652; l1P / ro[2]:1.990000
[ True  True  True]
[  1.59962165e+08   1.68501336e+08   1.61528564e+08]
[ True False False]
[ 1161565.14765666 -7377605.59040526  -404833.49337983]
 
l1P: 161933397.3519; l1P / ro[2]:2.000000
[ True  True  True]
[  1.60366999e+08   1.68906169e+08   1.61933397e+08]
[ True False  True]
[ 1566398.6410365  -6972772.09702542        0.        ]
 
l1P: 175697736.1268; l1P / ro[2]:2.170000
[ True  True  True]
[  1.67249168e+08   1.75788339e+08   1.68815567e+08]
[ True False  True]
[ 8448568.02849357   -90602.70956835  6882169.38745707]
 
l1P: 176507403.1136; l1P / ro[2]:2.180000
[ True  True  True]
[  1.67654002e+08   1.76193172e+08   1.69220400e+08]
[ True  True  True]
[ 8853401.52187341   314230.78381149  7287002.88083692]
 


***QUIZ QUESTION***

What range of values of `l1_penalty` would set **both** `w[1]` and `w[2]` to zero, if we were to take a step in that coordinate? 

In [55]:
searchL1P(ro[2] * np.array([1.0 / 3, 1.0 / 2, 1.0, 2.0, 3.0]))

l1P: 26988899.5587; l1P / ro[2]:0.333333
[ True  True  True]
[  9.28947498e+07   1.01433921e+08   9.44611485e+07]
[False False False]
[-65905850.25560155 -74445020.99366346 -67472248.89663804]
 
l1P: 40483349.3380; l1P / ro[2]:0.500000
[ True  True  True]
[  9.96419747e+07   1.08181145e+08   1.01208373e+08]
[False False False]
[-59158625.36593774 -67697796.10399966 -60725024.00697424]
 
l1P: 80966698.6760; l1P / ro[2]:1.000000
[ True  True  True]
[  1.19883649e+08   1.28422820e+08   1.21450048e+08]
[False False False]
[-38916950.69694633 -47456121.43500825 -40483349.33798283]
 
l1P: 161933397.3519; l1P / ro[2]:2.000000
[ True  True  True]
[  1.60366999e+08   1.68906169e+08   1.61933397e+08]
[ True False  True]
[ 1566398.6410365  -6972772.09702542        0.        ]
 
l1P: 242900096.0279; l1P / ro[2]:3.000000
[ True  True  True]
[  2.00850348e+08   2.09389519e+08   2.02416747e+08]
[ True  True  True]
[ 42049747.97901931  33510577.24095739  40483349.33798282]
 


In [58]:
searchL1P(ro[2] * np.array([1.0, 2.0, 3.0, 4.0]))

l1P: 80966698.6760; l1P / ro[2]:1.000000
[ True  True  True]
[  1.19883649e+08   1.28422820e+08   1.21450048e+08]
[False False False]
[-38916950.69694633 -47456121.43500825 -40483349.33798283]
 
l1P: 161933397.3519; l1P / ro[2]:2.000000
[ True  True  True]
[  1.60366999e+08   1.68906169e+08   1.61933397e+08]
[ True False  True]
[ 1566398.6410365  -6972772.09702542        0.        ]
 
l1P: 242900096.0279; l1P / ro[2]:3.000000
[ True  True  True]
[  2.00850348e+08   2.09389519e+08   2.02416747e+08]
[ True  True  True]
[ 42049747.97901931  33510577.24095739  40483349.33798282]
 
l1P: 323866794.7039; l1P / ro[2]:4.000000
[ True  True  True]
[  2.41333697e+08   2.49872868e+08   2.42900096e+08]
[ True  True  True]
[ 82533097.31700215  73993926.57894023  80966698.67596565]
 


In [59]:
searchL1P(ro[2] * np.array([2.0, 2.5, 3.0, 4.0, 5.0]))

l1P: 161933397.3519; l1P / ro[2]:2.000000
[ True  True  True]
[  1.60366999e+08   1.68906169e+08   1.61933397e+08]
[ True False  True]
[ 1566398.6410365  -6972772.09702542        0.        ]
 
l1P: 202416746.6899; l1P / ro[2]:2.500000
[ True  True  True]
[  1.80608673e+08   1.89147844e+08   1.82175072e+08]
[ True  True  True]
[ 21808073.31002791  13268902.57196599  20241674.66899142]
 
l1P: 242900096.0279; l1P / ro[2]:3.000000
[ True  True  True]
[  2.00850348e+08   2.09389519e+08   2.02416747e+08]
[ True  True  True]
[ 42049747.97901931  33510577.24095739  40483349.33798282]
 
l1P: 323866794.7039; l1P / ro[2]:4.000000
[ True  True  True]
[  2.41333697e+08   2.49872868e+08   2.42900096e+08]
[ True  True  True]
[ 82533097.31700215  73993926.57894023  80966698.67596565]
 
l1P: 404833493.3798; l1P / ro[2]:5.000000
[ True  True  True]
[  2.81817047e+08   2.90356217e+08   2.83383445e+08]
[ True  True  True]
[  1.23016447e+08   1.14477276e+08   1.21450048e+08]
 


In [60]:
searchL1P(ro[2] * np.array([2.0, 2.2, 2.5, 3.0, 5.0, 9.0]))

l1P: 161933397.3519; l1P / ro[2]:2.000000
[ True  True  True]
[  1.60366999e+08   1.68906169e+08   1.61933397e+08]
[ True False  True]
[ 1566398.6410365  -6972772.09702542        0.        ]
 
l1P: 178126737.0871; l1P / ro[2]:2.200000
[ True  True  True]
[  1.68463669e+08   1.77002839e+08   1.70030067e+08]
[ True  True  True]
[ 9663068.50863306  1123897.77057114  8096669.86759657]
 
l1P: 202416746.6899; l1P / ro[2]:2.500000
[ True  True  True]
[  1.80608673e+08   1.89147844e+08   1.82175072e+08]
[ True  True  True]
[ 21808073.31002791  13268902.57196599  20241674.66899142]
 
l1P: 242900096.0279; l1P / ro[2]:3.000000
[ True  True  True]
[  2.00850348e+08   2.09389519e+08   2.02416747e+08]
[ True  True  True]
[ 42049747.97901931  33510577.24095739  40483349.33798282]
 
l1P: 404833493.3798; l1P / ro[2]:5.000000
[ True  True  True]
[  2.81817047e+08   2.90356217e+08   2.83383445e+08]
[ True  True  True]
[  1.23016447e+08   1.14477276e+08   1.21450048e+08]
 
l1P: 728700288.0837; l1P / ro[2]

In [62]:
searchL1P(ro[2] * np.array([2.0, 2.1, 2.2, 3.0, 9.0, 14.0]))

l1P: 161933397.3519; l1P / ro[2]:2.000000
[ True  True  True]
[  1.60366999e+08   1.68906169e+08   1.61933397e+08]
[ True False  True]
[ 1566398.6410365  -6972772.09702542        0.        ]
 
l1P: 170030067.2195; l1P / ro[2]:2.100000
[ True  True  True]
[  1.64415334e+08   1.72954504e+08   1.65981732e+08]
[ True False  True]
[ 5614733.57483478 -2924437.16322714  4048334.93379828]
 
l1P: 178126737.0871; l1P / ro[2]:2.200000
[ True  True  True]
[  1.68463669e+08   1.77002839e+08   1.70030067e+08]
[ True  True  True]
[ 9663068.50863306  1123897.77057114  8096669.86759657]
 
l1P: 242900096.0279; l1P / ro[2]:3.000000
[ True  True  True]
[  2.00850348e+08   2.09389519e+08   2.02416747e+08]
[ True  True  True]
[ 42049747.97901931  33510577.24095739  40483349.33798282]
 
l1P: 728700288.0837; l1P / ro[2]:9.000000
[ True  True  True]
[  4.43750444e+08   4.52289615e+08   4.45316843e+08]
[ True  True  True]
[  2.84949844e+08   2.76410673e+08   2.83383445e+08]
 
l1P: 1133533781.4635; l1P / ro[2]:1

In [63]:
searchL1P(ro[2] * np.array([2.1, 2.15, 2.2, 3.0, 14.0, 23.0]))

l1P: 170030067.2195; l1P / ro[2]:2.100000
[ True  True  True]
[  1.64415334e+08   1.72954504e+08   1.65981732e+08]
[ True False  True]
[ 5614733.57483478 -2924437.16322714  4048334.93379828]
 
l1P: 174078402.1533; l1P / ro[2]:2.150000
[ True  True  True]
[  1.66439501e+08   1.74978672e+08   1.68005900e+08]
[ True False  True]
[ 7638901.04173392  -900269.696328    6072502.40069743]
 
l1P: 178126737.0871; l1P / ro[2]:2.200000
[ True  True  True]
[  1.68463669e+08   1.77002839e+08   1.70030067e+08]
[ True  True  True]
[ 9663068.50863306  1123897.77057114  8096669.86759657]
 
l1P: 242900096.0279; l1P / ro[2]:3.000000
[ True  True  True]
[  2.00850348e+08   2.09389519e+08   2.02416747e+08]
[ True  True  True]
[ 42049747.97901931  33510577.24095739  40483349.33798282]
 
l1P: 1133533781.4635; l1P / ro[2]:14.000000
[ True  True  True]
[  6.46167191e+08   6.54706362e+08   6.47733589e+08]
[ True  True  True]
[  4.87366591e+08   4.78827420e+08   4.85800192e+08]
 
l1P: 1862234069.5472; l1P / ro[2]

So we can say that `ro[i]` quantifies the significance of the i-th feature: the larger `ro[i]` is, the more likely it is for the i-th feature to be retained.

## Single Coordinate Descent Step

Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.

In [64]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, \
                                  l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix, weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = np.sum(feature_matrix[:,i] * \
                   (output - prediction + \
                    weights[i] * feature_matrix[:,i]))

    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty / 2.
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - l1_penalty / 2.
    else:
        new_weight_i = 0.
    
    return new_weight_i

To test the function, run the following cell:

In [65]:
# should print 0.425558846691
import math
print lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]]), 
                                   np.array([1., 1.]), np.array([1., 4.]), 0.1)

0.425558846691


## Cyclical coordinate descent 

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:
1. As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
2. After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to step 1.

Return weights

**IMPORTANT: when computing a new weight for coordinate i, make sure to incorporate the new weights for coordinates 0, 1, ..., i-1. One good way is to update your weights variable in-place. See following pseudocode for illustration.**
```
for i in range(len(weights)):
    old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
    # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
    #     and old values for weight[i], ..., weight[d-1]
    weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
    
    # use old_weights_i to compute change in coordinate
    ...
```

In [143]:
print 2 % 10

2


In [150]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, \
                                      initial_weights, l1_penalty, \
                                      tolerance):
    converged = False
    old_weights = np.array(initial_weights)
    iterNum = 0
    iterMax = 100
    while not converged:
        iterNum += 1;
        weights = old_weights
        if (iterNum < 10) or (iterNum % 10 == 1):
            print "iteration %d: weights: %s" % \
                (iterNum, (np.vectorize("%.4e".__mod__)(old_weights)))

        if (iterNum > iterMax):
            print "iteration exceeds max iterations:%d" % (iterMax) 
            
        diff_weights = np.zeros(len(weights))    
        for i in range(len(weights)):
            old_weights_i = weights[i]
            weights[i] = lasso_coordinate_descent_step(i, feature_matrix, \
                                            output, weights, l1_penalty)
            diff_weights[i] = weights[i] - old_weights_i
                
        if np.absolute(diff_weights).max() < tolerance:
            converged = True
            
        old_weigths = weights    
                            
    if converged:
        print "converged weights: %s" % \
            (np.vectorize("%.4e".__mod__)(weights))
            
    return weights

Using the following parameters, learn the weights on the sales dataset. 

In [125]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

First create a normalized version of the feature matrix, `normalized_simple_feature_matrix`

In [126]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, \
                                                 my_output)
(normalized_simple_feature_matrix, simple_norms) = \
    normalize_features(simple_feature_matrix) # normalize features

normalize_features: norms:
[  1.47013605e+02   3.34257264e+05   5.14075870e+02]


Then, run your implementation of LASSO coordinate descent:

In [127]:
#print np.savetxt(weights, ",".join(["%.4e"] * weights.shape[0]) 
#print 'weights: %s' % (np.vectorize("%.4e".__mod__)(weights))
print 'initial_weights: %s' % \
    (np.vectorize("%.4e".__mod__)(initial_weights))

initial_weights: ['0.0000e+00' '0.0000e+00' '0.0000e+00']


In [128]:
weights = \
    lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, \
                            output, initial_weights, l1_penalty, tolerance)

iteration 1: weights: ['0.0000e+00' '0.0000e+00' '0.0000e+00']
iteration 2: weights: ['7.9400e+07' '1.0305e+07' '-2.9972e+05']
iteration 3: weights: ['7.0262e+07' '1.8948e+07' '0.0000e+00']
iteration 4: weights: ['6.2067e+07' '2.6161e+07' '0.0000e+00']
iteration 5: weights: ['5.5468e+07' '3.2198e+07' '0.0000e+00']
iteration 6: weights: ['4.9946e+07' '3.7249e+07' '0.0000e+00']
iteration 7: weights: ['4.5325e+07' '4.1477e+07' '0.0000e+00']
iteration 8: weights: ['4.1458e+07' '4.5014e+07' '0.0000e+00']
iteration 9: weights: ['3.8222e+07' '4.7975e+07' '0.0000e+00']
iteration 10: weights: ['3.5514e+07' '5.0452e+07' '0.0000e+00']
iteration 11: weights: ['3.3248e+07' '5.2525e+07' '0.0000e+00']
iteration 12: weights: ['3.1351e+07' '5.4260e+07' '0.0000e+00']
iteration 13: weights: ['2.9764e+07' '5.5712e+07' '0.0000e+00']
iteration 14: weights: ['2.8436e+07' '5.6927e+07' '0.0000e+00']
iteration 15: weights: ['2.7325e+07' '5.7943e+07' '0.0000e+00']
iteration 16: weights: ['2.6395e+07' '5.8794e+07

***QUIZ QUESTIONS***
1. What is the RSS of the learned model on the normalized dataset?
2. Which features had weight zero at convergence?

[[ 147095.21879915  222958.65851223       0.        ]
 [ 147095.21879915  485596.40031902       0.        ]
 [ 147095.21879915  145489.97208002       0.        ]
 ..., 
 [ 147095.21879915  192726.9760021        0.        ]
 [ 147095.21879915  302316.82510133       0.        ]
 [ 147095.21879915  192726.9760021        0.        ]]


In [131]:
print 'normalized dataset RSS: %.4e' % \
    ((output - \
      predict_output(normalized_simple_feature_matrix, weights)) ** 
     2).sum()    

normalized dataset RSS: 1.6305e+15


In [135]:
print 'features:%s' % (['(intercept)'] + simple_features)
print 'weights: %s' % (np.vectorize("%.4e".__mod__)(weights))

features:['(intercept)', 'sqft_living', 'bedrooms']
weights: ['2.1625e+07' '6.3157e+07' '0.0000e+00']


# Evaluating LASSO fit with more features

Let us split the sales dataset into training and test sets.

In [136]:
train_data,test_data = sales.random_split(.8,seed=0)

Let us consider the following set of features.

In [137]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

First, create a normalized feature matrix from the TRAINING data with these features.  (Make you store the norms for the normalization, since we'll use them later)

In [139]:
all_features_train_matrix, train_output = \
    get_numpy_data(train_data, all_features, my_output)
all_features_train_norm_matrix, all_features_norms = \
    normalize_features(all_features_train_matrix)

normalize_features: norms:
[  1.31848398e+02   4.60040216e+02   2.96850552e+02   2.99962419e+05
   5.81709718e+06   2.04107815e+02   1.15325626e+01   1.05933942e+02
   4.57793622e+02   1.02101959e+03   2.59726472e+05   7.01224951e+04
   2.59922094e+05   5.36953839e+04]


First, learn the weights with `l1_penalty=1e7`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e7`, you will need them later.

In [145]:
initial_weights = np.zeros(len(all_features) + 1)
l1_penalty = 1e7
tolerance = 1.0
weights1e7 = \
    lasso_cyclical_coordinate_descent(all_features_train_norm_matrix, \
                    train_output, initial_weights, l1_penalty, tolerance)

iteration 1: weights: ['0.0000e+00' '0.0000e+00' '0.0000e+00' '0.0000e+00' '0.0000e+00'
 '0.0000e+00' '0.0000e+00' '0.0000e+00' '0.0000e+00' '0.0000e+00'
 '0.0000e+00' '0.0000e+00' '0.0000e+00' '0.0000e+00']
iteration 11: weights: ['3.1841e+07' '0.0000e+00' '4.9289e+06' '3.6586e+07' '0.0000e+00'
 '0.0000e+00' '3.2998e+06' '7.9532e+06' '0.0000e+00' '0.0000e+00'
 '0.0000e+00' '0.0000e+00' '0.0000e+00' '0.0000e+00']
iteration 21: weights: ['2.5625e+07' '0.0000e+00' '0.0000e+00' '4.7263e+07' '0.0000e+00'
 '0.0000e+00' '3.3159e+06' '7.4037e+06' '0.0000e+00' '0.0000e+00'
 '0.0000e+00' '0.0000e+00' '0.0000e+00' '0.0000e+00']
iteration 31: weights: ['2.4646e+07' '0.0000e+00' '0.0000e+00' '4.8185e+07' '0.0000e+00'
 '0.0000e+00' '3.3172e+06' '7.3433e+06' '0.0000e+00' '0.0000e+00'
 '0.0000e+00' '0.0000e+00' '0.0000e+00' '0.0000e+00']
iteration 41: weights: ['2.4469e+07' '0.0000e+00' '0.0000e+00' '4.8352e+07' '0.0000e+00'
 '0.0000e+00' '3.3175e+06' '7.3324e+06' '0.0000e+00' '0.0000e+00'
 '0.0000e+

***QUIZ QUESTION***

What features had non-zero weight in this case?

In [148]:
#print 'features:%s' % (['(intercept)'] + all_features)
#print 'weights: %s' % (np.vectorize("%.4e".__mod__)(weights1e7))
weights1e7Coeffs = \
    pd.DataFrame({'feature': ['(intercept)'] + all_features, 
                  'weight':  weights1e7})
print weights1e7Coeffs    

features:['(intercept)', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated']
weights: ['2.4430e+07' '0.0000e+00' '0.0000e+00' '4.8389e+07' '0.0000e+00'
 '0.0000e+00' '3.3175e+06' '7.3300e+06' '0.0000e+00' '0.0000e+00'
 '0.0000e+00' '0.0000e+00' '0.0000e+00' '0.0000e+00']
          feature           weight
0     (intercept)  24429600.609333
1        bedrooms         0.000000
2       bathrooms         0.000000
3     sqft_living  48389174.352280
4        sqft_lot         0.000000
5          floors         0.000000
6      waterfront   3317511.162720
7            view   7329961.984896
8       condition         0.000000
9           grade         0.000000
10     sqft_above         0.000000
11  sqft_basement         0.000000
12       yr_built         0.000000
13   yr_renovated         0.000000


Next, learn the weights with `l1_penalty=1e8`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e8`, you will need them later.

In [151]:
initial_weights = np.zeros(len(all_features) + 1)
l1_penalty = 1e8
tolerance = 1.0
weights1e8 = \
    lasso_cyclical_coordinate_descent(all_features_train_norm_matrix, \
                    train_output, initial_weights, l1_penalty, tolerance)

iteration 1: weights: ['0.0000e+00' '0.0000e+00' '0.0000e+00' '0.0000e+00' '0.0000e+00'
 '0.0000e+00' '0.0000e+00' '0.0000e+00' '0.0000e+00' '0.0000e+00'
 '0.0000e+00' '0.0000e+00' '0.0000e+00' '0.0000e+00']
iteration 2: weights: ['7.1115e+07' '0.0000e+00' '0.0000e+00' '0.0000e+00' '0.0000e+00'
 '0.0000e+00' '0.0000e+00' '0.0000e+00' '0.0000e+00' '0.0000e+00'
 '0.0000e+00' '0.0000e+00' '0.0000e+00' '0.0000e+00']
converged weights: ['7.1115e+07' '0.0000e+00' '0.0000e+00' '0.0000e+00' '0.0000e+00'
 '0.0000e+00' '0.0000e+00' '0.0000e+00' '0.0000e+00' '0.0000e+00'
 '0.0000e+00' '0.0000e+00' '0.0000e+00' '0.0000e+00']


***QUIZ QUESTION***

What features had non-zero weight in this case?

In [152]:
weights1e8Coeffs = \
    pd.DataFrame({'feature': ['(intercept)'] + all_features, 
                  'weight':  weights1e8})
print weights1e8Coeffs  

          feature           weight
0     (intercept)  71114625.752809
1        bedrooms         0.000000
2       bathrooms         0.000000
3     sqft_living         0.000000
4        sqft_lot         0.000000
5          floors         0.000000
6      waterfront         0.000000
7            view         0.000000
8       condition         0.000000
9           grade         0.000000
10     sqft_above         0.000000
11  sqft_basement         0.000000
12       yr_built         0.000000
13   yr_renovated         0.000000


Finally, learn the weights with `l1_penalty=1e4`, on the training data. Initialize weights to all zeros, and set the `tolerance=5e5`.  Call resulting weights `weights1e4`, you will need them later.  (This case will take quite a bit longer to converge than the others above.)

In [153]:
initial_weights = np.zeros(len(all_features) + 1)
l1_penalty = 1e4
tolerance = 5e5
weights1e4 = \
    lasso_cyclical_coordinate_descent(all_features_train_norm_matrix, \
                    train_output, initial_weights, l1_penalty, tolerance)

iteration 1: weights: ['0.0000e+00' '0.0000e+00' '0.0000e+00' '0.0000e+00' '0.0000e+00'
 '0.0000e+00' '0.0000e+00' '0.0000e+00' '0.0000e+00' '0.0000e+00'
 '0.0000e+00' '0.0000e+00' '0.0000e+00' '0.0000e+00']
iteration 2: weights: ['7.1115e+07' '3.9564e+06' '4.9634e+06' '5.3518e+06' '-1.0298e+06'
 '-8.7702e+06' '1.2531e+07' '1.0819e+07' '-8.8522e+06' '3.6910e+06'
 '6.5527e+06' '2.9464e+06' '-1.1588e+07' '3.0435e+06']
iteration 3: weights: ['7.0458e+07' '6.4254e+06' '8.5479e+06' '8.9149e+06' '-1.4502e+06'
 '-1.3686e+07' '7.6000e+06' '1.2240e+07' '-1.2644e+07' '6.6084e+06'
 '1.1850e+07' '3.4267e+06' '-2.0052e+07' '3.1924e+06']
iteration 4: weights: ['7.0403e+07' '8.0298e+06' '1.1726e+07' '1.1909e+07' '-1.6470e+06'
 '-1.7274e+07' '6.6377e+06' '1.1699e+07' '-1.5634e+07' '8.9070e+06'
 '1.6034e+07' '3.5746e+06' '-2.6870e+07' '3.1068e+06']
iteration 5: weights: ['7.0415e+07' '8.9947e+06' '1.4529e+07' '1.4487e+07' '-1.7437e+06'
 '-1.9900e+07' '6.5636e+06' '1.0978e+07' '-1.8097e+07' '1.0686e+07'

***QUIZ QUESTION***

What features had non-zero weight in this case?

In [154]:
weights1e4Coeffs = \
    pd.DataFrame({'feature': ['(intercept)'] + all_features, 
                  'weight':  weights1e4})
print weights1e4Coeffs  

          feature           weight
0     (intercept)  77779073.912652
1        bedrooms -22884012.250234
2       bathrooms  15348487.080900
3     sqft_living  92166869.698831
4        sqft_lot  -2139328.082428
5          floors  -8818455.544095
6      waterfront   6494209.733107
7            view   7065162.050532
8       condition   4119079.210068
9           grade  18436483.526188
10     sqft_above -14566678.545143
11  sqft_basement  -5528348.751794
12       yr_built -83591746.207305
13   yr_renovated   2784276.460129


## Rescaling learned weights

Recall that we normalized our feature matrix, before learning the weights.  To use these weights on a test set, we must normalize the test data in the same way.

Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data: 

In this case, we must scale the resulting weights so that we can make predictions with *original* features:
 1. Store the norms of the original features to a vector called `norms`:
```
features, norms = normalize_features(features)
```
 2. Run Lasso on the normalized features and obtain a `weights` vector
 3. Compute the weights for the original features by performing element-wise division, i.e.
```
weights_normalized = weights / norms
```
Now, we can apply `weights_normalized` to the test data, without normalizing it!

Create a normalized version of each of the weights learned above. (`weights1e4`, `weights1e7`, `weights1e8`).

In [155]:
weights1e4Normalized = weights1e4 / all_features_norms
weights1e7Normalized = weights1e7 / all_features_norms
print weights1e7Normalized[3]
weights1e8Normalized = weights1e8 / all_features_norms

161.317456248


To check your results, if you call `normalized_weights1e7` the normalized version of `weights1e7`, then:
```
print normalized_weights1e7[3]
```
should return 161.31745624837794.

## Evaluating each of the learned models on the test data

Let's now evaluate the three models on the test data:

In [156]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, \
                                                    all_features, 'price')

Compute the RSS of each of the three normalized weights on the (unnormalized) `test_feature_matrix`:

In [166]:
print predict_output(test_feature_matrix, weights1e4Normalized).shape
print test_output.shape
print ((test_output - \
        predict_output(test_feature_matrix, weights1e4Normalized)) ** \
       2).sum()

(4229,)
(4229,)
2.2778100476e+14


In [167]:
def getTestRSS(weightsNormalized):
    return ((test_output - \
             predict_output(test_feature_matrix, weightsNormalized)) ** 
            2).sum()  

In [170]:
print 'weights1e4Normalized Test RSS:%.4e' % \
    getTestRSS(weights1e4Normalized)
print 'weights1e7Normalized Test RSS:%.4e' % \
    getTestRSS(weights1e7Normalized)
print 'weights1e8Normalized Test RSS:%.4e' % \
    getTestRSS(weights1e8Normalized)    

weights1e4Normalized Test RSS:2.2778e+14
weights1e7Normalized Test RSS:2.7596e+14
weights1e8Normalized Test RSS:5.3717e+14


***QUIZ QUESTION***

Which model performed best on the test data?